In [2]:
import numpy as np
from scipy import stats 
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker

In [3]:
engine = create_engine("postgresql:///music", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
con = engine.connect()

2019-05-08 08:53:36,505 INFO sqlalchemy.engine.base.Engine select version()
2019-05-08 08:53:36,507 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,509 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-08 08:53:36,510 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,512 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-08 08:53:36,512 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,514 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-08 08:53:36,515 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,516 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-05-08 08:53:36,516 INFO sqlalchemy.engine.base.Engine {}


In [11]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE pub_date NOT IN (SELECT pub_date FROM reviews WHERE pub_date < '2014-01-01' AND pub_date > '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]
len(result)

2019-05-08 08:57:14,900 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE pub_date NOT IN (SELECT pub_date FROM reviews WHERE pub_date < '2014-01-01' AND pub_date > '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 08:57:14,901 INFO sqlalchemy.engine.base.Engine {}


715

In [20]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]
len(result)

2019-05-08 09:14:07,882 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 09:14:07,883 INFO sqlalchemy.engine.base.Engine {}


715

In [27]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date < '2010-01-01'
        GROUP BY artist
        ORDER BY avg(score))
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]
len(result)

2019-05-08 09:31:47,517 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date < '2010-01-01'
        GROUP BY artist
        ORDER BY avg(score))
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 09:31:47,518 INFO sqlalchemy.engine.base.Engine {}


715